In [1]:
DROP TABLE IF EXISTS Wypozyczenie;

DROP TABLE IF EXISTS Egzemplarz;

DROP TABLE IF EXISTS Czytelnik;

DROP TABLE IF EXISTS Ksiazka;

CREATE TABLE Ksiazka
( Ksiazka_ID INT IDENTITY
, ISBN VARCHAR(20)
, Tytul VARCHAR(300)
, Autor VARCHAR(200)
, Rok_Wydania INT
, Cena DECIMAL(10,2)
, Wypozyczona_Ostatni_Miesiac BIT
, CONSTRAINT Ksiazka_PK PRIMARY KEY (Ksiazka_ID)
, CONSTRAINT Ksiazka_UK_ISBN UNIQUE (ISBN)
);

CREATE TABLE Egzemplarz
( Egzemplarz_ID INT IDENTITY
, Sygnatura CHAR(8)
, Ksiazka_ID INT
, CONSTRAINT Egzemplarz_PK PRIMARY KEY (Egzemplarz_ID)
, CONSTRAINT Egzemplarz_UK_Sygnatura UNIQUE (Sygnatura)
, CONSTRAINT Egzemplarz_FK FOREIGN KEY (Ksiazka_ID) REFERENCES Ksiazka (Ksiazka_ID) ON DELETE CASCADE
);

CREATE TABLE Czytelnik
( Czytelnik_ID INT IDENTITY
, PESEL CHAR(11)
, Nazwisko VARCHAR(30)
, Miasto VARCHAR(30)
, Data_Urodzenia DATE
, Ostatnie_Wypozyczenie DATE
, CONSTRAINT Czytelnik_PK PRIMARY KEY (Czytelnik_ID)
, CONSTRAINT Czytelnik_UK_PESEL UNIQUE (PESEL)
);

CREATE TABLE Wypozyczenie
( Wypozyczenie_ID INT IDENTITY
, Czytelnik_ID INT
, Egzemplarz_ID INT
, Data DATE
, Liczba_Dni INT
, CONSTRAINT Wypozyczenie_PK PRIMARY KEY (Wypozyczenie_ID)
, CONSTRAINT Wypozyczenie_FK_Czytelnik FOREIGN KEY (Czytelnik_ID) REFERENCES Czytelnik (Czytelnik_ID) ON DELETE CASCADE
, CONSTRAINT Wypozyczenie_FK_Egzemplarz FOREIGN KEY (Egzemplarz_ID) REFERENCES Egzemplarz (Egzemplarz_ID) ON DELETE CASCADE
);

SET IDENTITY_INSERT Ksiazka ON
INSERT INTO Ksiazka (Ksiazka_ID,ISBN,Tytul,Autor,Rok_Wydania,Cena) VALUES
(1,'83-246-0279-8','Microsoft Access. Podrêcznik administratora','Helen Feddema',2006,69),
(2,'83-246-0653-X','SQL Server 2005. Programowanie. Od podstaw','Robert Vieira',2007,97),
(3,'978-83-246-0549-1','SQL Server 2005. Wycinij wszystko','Eric L. Brown',2007,57),
(4,'978-83-246-1258-1','PHP, MySQL i MVC. Tworzenie witryn WWW opartych na bazie danych','W³odzimierz Gajda',2010,79),
(5,'978-83-246-2060-9','Access 2007 PL. Seria praktyk','Andrew Unsworth',2009,39),
(6,'978-83-246-2188-0','Czysty kod. Podrêcznik dobrego programisty','Robert C. Martin',2010,67);
SET IDENTITY_INSERT Ksiazka OFF

SET IDENTITY_INSERT Egzemplarz ON
INSERT INTO Egzemplarz (Egzemplarz_ID,Ksiazka_ID,Sygnatura) VALUES
(1,5,'S0001'),
(2,5,'S0002'),
(3,1,'S0003'),
(4,1,'S0004'),
(5,1,'S0005'),
-- (6,2,'S0006'),
(7,3,'S0007'),
(8,3,'S0008'),
(9,3,'S0009'),
(10,3,'S0010'),
(11,6,'S0011'),
(12,6,'S0012'),
(13,4,'S0013'),
(14,4,'S0014'),
(15,4,'S0015');
SET IDENTITY_INSERT Egzemplarz OFF

SET IDENTITY_INSERT Czytelnik ON
INSERT INTO Czytelnik (CZYTELNIK_ID,PESEL,NAZWISKO,MIASTO,DATA_URODZENIA) VALUES
(1,'55101011111','Kowalski','Wroc³aw','1955-10-10'),
(2,'60101033333','Maliniak','Wroc³aw','1960-10-10'),
(3,'65120122222','Nowak','Warszawa','1965-12-01');
SET IDENTITY_INSERT Czytelnik OFF

SET IDENTITY_INSERT Wypozyczenie ON
INSERT INTO Wypozyczenie (Wypozyczenie_ID,Czytelnik_ID,Egzemplarz_ID,Data,Liczba_Dni) VALUES
(1,1,3,'2020-02-01',12),
(2,1,4,'2020-01-05',20),
(3,1,15,'2020-01-21',45),
(4,2,8,'2020-01-13',7),
(5,3,4,'2020-02-01',14),
(6,3,12,'2020-02-02',10),
(7,3,12,'2020-02-12',3),
(8,3,12,'2020-02-16',4),
(9,1,12,'2020-02-20',2),
(10,2,12,'2020-02-22',5),
(11,2,12,'2020-02-28',12),
(12,1,12,'2020-03-10',8),
(13,3,12,'2020-03-15',4);
SET IDENTITY_INSERT Wypozyczenie OFF

(6 rows affected)

(14 rows affected)

(3 rows affected)

(13 rows affected)

Total execution time: 00:00:01.022

# Task 1

Create a T-SQL function taking as an input a number of days and returning in result a table (PESEL, specimens number) which consists of readers list keeping at least one specimen not shorter that the number of days provided as the input. In the second result column please provide the number of all specimens currently being hold by a reader.

In [40]:
DROP FUNCTION IF EXISTS task1

Commands completed successfully.

Total execution time: 00:00:00.061

In [41]:
-- CREATE FUNCTION task1 (@days_no int) RETURNS TABLE(PESEL CHAR(11), speciments INT)
CREATE FUNCTION task1 (@days_no INT) RETURNS TABLE
AS
RETURN
    SELECT c.PESEL, COUNT(w.Wypozyczenie_ID) as speciments
    FROM dbo.Czytelnik c, dbo.Wypozyczenie w
    WHERE DATEDIFF(day, w.[Data], cast(getdate() as Date)) >= @days_no
    GROUP BY c.PESEL, c.Czytelnik_ID

Commands completed successfully.

Total execution time: 00:00:00.043

In [5]:
SELECT * FROM task1(100)

(3 rows affected)

Total execution time: 00:00:00.036

PESEL,speciments
55101011111,13
60101033333,13
65120122222,13


# Task 2

Create tables firstnames(PK id, firstname), lastnames(PK id, lastname) and fldata(PK firstname, PK lastname) where PK indicates the columns which should constitute the primary keys. Generate test data into tables firstnames and lastnames, then create a procedure which takes n as an input and in result (1) removes the current content, (2) inserts n random pairs (firstname, lastname) into the table fldata. In case n is larger than the number all possible pairs, an appropriate error should be communicate using THROW. Moreover, as primary key is defined on both columns (firstname,lastname) generation procedure needs to ensure that each pair is generated only once.

In [6]:
-- create tables
DROP TABLE IF EXISTS firstnames
DROP TABLE IF EXISTS lastnames
DROP TABLE IF EXISTS fldata

CREATE TABLE firstnames (
    id INT IDENTITY,
    firstname VARCHAR(30),
    CONSTRAINT Firstnames_id_PK PRIMARY KEY (id),
);

CREATE TABLE lastnames (
    id INT IDENTITY,
    lastname VARCHAR(30),
    CONSTRAINT Lastnames_id_PK PRIMARY KEY (id),
);

CREATE TABLE fldata (
    firstname VARCHAR(30),
    lastname VARCHAR(30),
    CONSTRAINT PK_Person_task2 PRIMARY KEY (firstname, lastname)
);

INSERT INTO firstnames (firstname) VALUES
('Karol'),
('Piotr'),
('Maciej'),
('Szymon'),
('Paweł'),
('Jacek'),
('Marcin'),
('Kamil'),
('Norbert'),
('Maciej'),
('Adam'),
('Dawid'),
('Daniel');

INSERT INTO lastnames (lastname) VALUES
('Guzikowski'),
('Mały'),
('Nowak'),
('Kowalski'),
('Łysek'),
('Kowalczyk'),
('Osiński'),
('Prus'),
('Sienkiewicz'),
('Ryba');

(13 rows affected)

(10 rows affected)

Total execution time: 00:00:00.104

In [11]:
DROP PROCEDURE IF EXISTS task2

Commands completed successfully.

Total execution time: 00:00:00.042

In [12]:
CREATE PROCEDURE task2 @n INT
AS
BEGIN
    DELETE FROM fldata
    DECLARE @i INT = 1, @ready_pairs INT = 0
    BEGIN TRY
        WHILE @ready_pairs <= @n
        BEGIN
            DECLARE @j INT = 1, @max INT
            SELECT @max = COUNT(lastname) FROM lastnames
            WHILE @ready_pairs <= @n AND @j <= @max
            BEGIN
                DECLARE @fname VARCHAR(30), @lname VARCHAR(30)
                SELECT @fname = firstname FROM firstnames WHERE id = @i
                SELECT @lname = lastname FROM lastnames WHERE id = @j
                INSERT INTO fldata (firstname, lastname) VALUES (@fname, @lname)
                SET @j = @j + 1
                SET @ready_pairs = @ready_pairs + 1
            END
            SET @i = @i + 1
        END
    END TRY
    BEGIN CATCH
        PRINT('n is too big!');
        THROW;
    END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.041

In [13]:
EXEC task2 @n = 10000
SELECT * FROM fldata

(41 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(0 rows affected)

n is too big!

: Msg 2627, Level 14, State 1, Procedure task2, Line 16
Violation of PRIMARY KEY constraint 'PK_Person_task2'. Cannot insert duplicate key in object 'dbo.fldata'. The duplicate key value is (Maciej, Guzikowski).

Total execution time: 00:00:00.555

# Task 3

Create procedure taking a table (czytelnik id) of readers IDs as an input and returning in result a table (reader id,sum days) which provides the total sum of number of days a specific reader has borrowed all specimens so far.

In [42]:
DROP TYPE IF EXISTS task3type

CREATE TYPE task3type AS TABLE (
    czytelnik_id INT
);

: Msg 3732, Level 16, State 1, Line 1
Cannot drop type 'task3type' because it is being referenced by object 'task3'. There may be other objects that reference this type.

Total execution time: 00:00:00.049

In [47]:
DROP FUNCTION IF EXISTS task3

Commands completed successfully.

Total execution time: 00:00:00.045

In [48]:
CREATE FUNCTION task3 (
    @TableType task3type READONLY
)
RETURNS @tablica TABLE(
    reader_id INT,
    sum_days INT
)
BEGIN
    DECLARE @idColumn INT
    SELECT @idColumn = min(czytelnik_id) from @TableType
    WHILE @idColumn IS NOT NULL
    BEGIN
        DECLARE @sum_days INT
        SELECT @sum_days = SUM(liczba_dni) FROM Wypozyczenie WHERE Czytelnik_ID = @idColumn
        INSERT INTO @tablica (reader_id, sum_days) VALUES (@idColumn, @sum_days)
        SELECT @idColumn = min(czytelnik_id) from @TableType WHERE czytelnik_id > @idColumn 
    END
    RETURN
END

Commands completed successfully.

Total execution time: 00:00:00.050

In [ ]:
DECLARE @task3type1 task3type

INSERT INTO @task3type1 (czytelnik_id) VALUES
(1),
(2),
(3);

SELECT * FROM task3(@task3type1)

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.058

reader_id,sum_days
1,87
2,24
3,35
